In [50]:
import os
import requests
import pandas as pd

In [51]:

token = "06a4cf02a31a3b9fbed177f26e40af46d84a255c"

In [8]:
# API endpoint
#footprint is the correct API interface
url = "https://lca.aau.dk/api/footprint/" 

token = token

headers = {
    'Authorization': f'Bearer {token}',  
    'Content-Type': 'application/json'   
}

all_footprints= []
next_page_url = url  

while next_page_url:
    # Make the GET request
    response = requests.get(next_page_url, headers=headers)

    if response.status_code == 200:
        data = response.json()        
        all_footprints.extend(data.get("results", data))  # Adjust based on API structure
        next_page_url = data.get("next")  # Check for the 'next' key in the response
        if not next_page_url:
            print("All pages retrieved successfully!")
    else:
        # Handle errors
        print(f"Failed. Status code: {response.status_code}")
        print("Error message:", response.text)
        break

# Now you have all the data
print(f"Total results retrieved: {len(all_footprints)}")
print(next_page_url)


Failed. Status code: 429
Error message: {"detail":"Request was throttled. Expected available in 3482 seconds."}
Total results retrieved: 0
https://lca.aau.dk/api/footprint/


In [9]:
all_footprint_codes = [item['flow_code'] for item in all_footprints]
unique_footprint_codes = set(all_footprint_codes)
len(unique_footprint_codes)

0

In [11]:
list_abbreviation = ["A_POWC",
"A_POWG",
"A_POWN",
"A_POWH",
"A_POWW",
"A_POWP",
"A_POWB",
"A_POWS",
"A_POWO",
"A_POWM",
] #From Bonsai excel file, codes of all electricity producing activities

In [12]:
# check if all abbreviations are in the list
for i in list_abbreviation:
    if i not in unique_footprint_codes:
        print(f"{i} not in the list")

A_POWC not in the list
A_POWG not in the list
A_POWN not in the list
A_POWH not in the list
A_POWW not in the list
A_POWP not in the list
A_POWB not in the list
A_POWS not in the list
A_POWO not in the list
A_POWM not in the list


In [66]:
#technically, we don't have to call teh API again, but simply filter all_footprints
electr_footprint = []
for item in all_footprints:
    if item['flow_code'] in list_abbreviation:
        electr_footprint.append(item)


In [67]:
electr_footprint[0]

{'id': 32552,
 'flow_code': 'A_POWC',
 'nace_related_code': '40.11.a',
 'description': 'Production of electricity by coal',
 'unit_reference': 'TJ',
 'region_code': 'AT',
 'value': 211.86083667813315,
 'unit_emission': 'tonnes CO2eq',
 'version': 'v1.0.0'}

In [68]:
len(electr_footprint) #325 the amount of entries in the bonsai excel, so we are not missing any

325

In [69]:
# Create a pandas DataFrame from the list of dictionaries
df = pd.DataFrame(electr_footprint)
df = df[['description', 'region_code', 'unit_emission', 'unit_reference', 'value', 'flow_code', 'id']]
print(df)

                                 description region_code unit_emission  \
0          Production of electricity by coal          AT  tonnes CO2eq   
1          Production of electricity by coal          AU  tonnes CO2eq   
2          Production of electricity by coal          BE  tonnes CO2eq   
3          Production of electricity by coal          BG  tonnes CO2eq   
4          Production of electricity by coal          BR  tonnes CO2eq   
..                                       ...         ...           ...   
320  Production of electricity by geothermal          US  tonnes CO2eq   
321  Production of electricity by geothermal          WA  tonnes CO2eq   
322  Production of electricity by geothermal          WE  tonnes CO2eq   
323  Production of electricity by geothermal          WF  tonnes CO2eq   
324  Production of electricity by geothermal          WL  tonnes CO2eq   

    unit_reference       value flow_code     id  
0               TJ  211.860837    A_POWC  32552  
1          

In [35]:
 # test dataframe with correct dimensions

test_data = [
    {'id': 32552,
 'flow_code': 'A_POWC',
 'nace_related_code': '40.11.a',
 'description': 'Production of electricity by coal',
 'unit_reference': 'TJ',
 'region_code': 'AT',
 'value': 211.86083667813315,
 'unit_emission': 'tonnes CO2eq',
 'version': 'v1.0.0'}, 

 {'id': 32552,
 'flow_code': 'A_POWC',
 'nace_related_code': '40.11.a',
 'description': 'Production of electricity by coal',
 'unit_reference': 'TJ',
 'region_code': 'BE',
 'value': 211.86083667813315,
 'unit_emission': 'tonnes CO2eq',
 'version': 'v1.0.0'}
 ]

df = pd.DataFrame(test_data)
df = df[['description', 'region_code', 'unit_emission', 'unit_reference', 'value', 'flow_code', 'id']]
df

,description,region_code,unit_emission,unit_reference,value,flow_code,id
0,Production of electricity by coal,AT,tonnes CO2eq,TJ,211.860837,A_POWC,32552
1,Production of electricity by coal,BE,tonnes CO2eq,TJ,211.860837,A_POWC,32552


In [36]:
#convert values from t CO2/TJ to kg CO2/kWh
conversion_factor = 0.0036
df['value'] = df['value'] * conversion_factor

df["unit"] = "kg CO2/kWh"
df = df.drop(columns = ["unit_emission", "unit_reference"])
df =df.rename(columns={"value": "emission factor"})

,description,region_code,value,flow_code,id,unit
0,Production of electricity by coal,AT,0.762699,A_POWC,32552,kg CO2/kWh
1,Production of electricity by coal,BE,0.762699,A_POWC,32552,kg CO2/kWh


In [37]:

dict = {("Production of electricity by coal", "AT"): 0.001,}



In [43]:
# direct emission factors, which have been manually looked up as they couldn't be retrieved easily from the API
direct_emission_dict = {   # Production of electricity by coal
    ("Production of electricity by coal", "AT"): 0.757,
    ("Production of electricity by coal", "AU"): 0.999,
    ("Production of electricity by coal", "BE"): 0.928,
    ("Production of electricity by coal", "BG"): 1.09,
    ("Production of electricity by coal", "BR"): 1.03,
    ("Production of electricity by coal", "CA"): 0.95,
    ("Production of electricity by coal", "CN"): 0.868,
    ("Production of electricity by coal", "CZ"): 1.03,
    ("Production of electricity by coal", "DE"): 0.893,
    ("Production of electricity by coal", "EE"): 0.745,
    ("Production of electricity by coal", "ES"): 0.915,
    ("Production of electricity by coal", "FI"): 0.868,
    ("Production of electricity by coal", "FR"): 0.925,
    ("Production of electricity by coal", "GB"): 0.894,
    ("Production of electricity by coal", "GR"): 1,
    ("Production of electricity by coal", "HR"): 0.896,
    ("Production of electricity by coal", "HU"): 1.07,
    ("Production of electricity by coal", "ID"): 1.01,
    ("Production of electricity by coal", "IE"): 0.954,
    ("Production of electricity by coal", "IN"): 0.925,
    ("Production of electricity by coal", "IT"): 0.897,
    ("Production of electricity by coal", "JP"): 0.821,
    ("Production of electricity by coal", "KR"): 0.905,
    ("Production of electricity by coal", "MX"): 1.06,
    ("Production of electricity by coal", "NL"): 0.788,
    ("Production of electricity by coal", "PL"): 0.853,
    ("Production of electricity by coal", "PT"): 0.892,
    ("Production of electricity by coal", "RO"): 1.08,
    ("Production of electricity by coal", "TR"): 0.959,
    ("Production of electricity by coal", "US"): 0.942,
    ("Production of electricity by coal", "WA"): 0.984,
    ("Production of electricity by coal", "WE"): 1.06,
    ("Production of electricity by coal", "WF"): 1.17,
    ("Production of electricity by coal", "WL"): 0.963,
    ("Production of electricity by coal", "WM"): 0.893,
    ("Production of electricity by coal", "ZA"): 1.05,
    
    # Production of electricity by gas
    ("Production of electricity by gas", "AT"): 0.432,
    ("Production of electricity by gas", "AU"): 0.596,
    ("Production of electricity by gas", "BE"): 0.402,
    ("Production of electricity by gas", "BG"): 1.08,
    ("Production of electricity by gas", "BR"): 0.47,
    ("Production of electricity by gas", "CA"): 0.606,
    ("Production of electricity by gas", "CN"): 0.473,
    ("Production of electricity by gas", "CZ"): 0.387,
    ("Production of electricity by gas", "DE"): 0.42,
    ("Production of electricity by gas", "EE"): 0.23,
    ("Production of electricity by gas", "ES"): 0.427,
    ("Production of electricity by gas", "FI"): 0.532,
    ("Production of electricity by gas", "FR"): 0.382,
    ("Production of electricity by gas", "GB"): 0.388,
    ("Production of electricity by gas", "GR"): 0.369,
    ("Production of electricity by gas", "HR"): 0.452,
    ("Production of electricity by gas", "HU"): 0.402,
    ("Production of electricity by gas", "ID"): 0.541,
    ("Production of electricity by gas", "IE"): 0.367,
    ("Production of electricity by gas", "IN"): 0.506,
    ("Production of electricity by gas", "IT"): 0.381,
    ("Production of electricity by gas", "JP"): 0.418,
    ("Production of electricity by gas", "KR"): 0.397,
    ("Production of electricity by gas", "LU"): 5.93,
    ("Production of electricity by gas", "MX"): 0.448,
    ("Production of electricity by gas", "NL"): 0.369,
    ("Production of electricity by gas", "NO"): 0.552,
    ("Production of electricity by gas", "PT"): 0.372,
    ("Production of electricity by gas", "RO"): 0.409,
    ("Production of electricity by gas", "RU"): 1.15,
    ("Production of electricity by gas", "SI"): 0.859,
    ("Production of electricity by gas", "SK"): 0.509,
    ("Production of electricity by gas", "TR"): 0.395,
    ("Production of electricity by gas", "US"): 0.436,
    ("Production of electricity by gas", "WA"): 0.49,
    ("Production of electricity by gas", "WE"): 0.508,
    ("Production of electricity by gas", "WF"): 0.496,
    ("Production of electricity by gas", "WL"): 0.533,
    ("Production of electricity by gas", "WM"): 0.569,
    
    # Production of electricity by petroleum and other oil derivatives
    ("Production of electricity by petroleum and other oil derivatives", "AT"): 0.696,
    ("Production of electricity by petroleum and other oil derivatives", "AU"): 0.75,
    ("Production of electricity by petroleum and other oil derivatives", "BE"): 1.08,
    ("Production of electricity by petroleum and other oil derivatives", "BG"): 1.14,
    ("Production of electricity by petroleum and other oil derivatives", "BR"): 0.685,
    ("Production of electricity by petroleum and other oil derivatives", "CA"): 0.897,
    ("Production of electricity by petroleum and other oil derivatives", "CH"): 0.697,
    ("Production of electricity by petroleum and other oil derivatives", "CN"): 1.05,
    ("Production of electricity by petroleum and other oil derivatives", "CY"): 0.733,
    ("Production of electricity by petroleum and other oil derivatives", "CZ"): 0.745,
    ("Production of electricity by petroleum and other oil derivatives", "DE"): 0.742,
    ("Production of electricity by petroleum and other oil derivatives", "DK"): 1.1,
    ("Production of electricity by petroleum and other oil derivatives", "EE"): 0.74,
    ("Production of electricity by petroleum and other oil derivatives", "ES"): 0.753,
    ("Production of electricity by petroleum and other oil derivatives", "FI"): 0.772,
    ("Production of electricity by petroleum and other oil derivatives", "FR"): 0.726,
    ("Production of electricity by petroleum and other oil derivatives", "GB"): 0.991,
    ("Production of electricity by petroleum and other oil derivatives", "GR"): 0.6,
    ("Production of electricity by petroleum and other oil derivatives", "HR"): 0.697,
    ("Production of electricity by petroleum and other oil derivatives", "HU"): 0.919,
    ("Production of electricity by petroleum and other oil derivatives", "ID"): 0.809,
    ("Production of electricity by petroleum and other oil derivatives", "IE"): 0.767,
    ("Production of electricity by petroleum and other oil derivatives", "IN"): 1.15,
    ("Production of electricity by petroleum and other oil derivatives", "IT"): 0.808,
    ("Production of electricity by petroleum and other oil derivatives", "JP"): 0.641,
    ("Production of electricity by petroleum and other oil derivatives", "KR"): 0.71,
    ("Production of electricity by petroleum and other oil derivatives", "MT"): 0.777,
    ("Production of electricity by petroleum and other oil derivatives", "MX"): 0.769,
    ("Production of electricity by petroleum and other oil derivatives", "NL"): 0.701,
    ("Production of electricity by petroleum and other oil derivatives", "NO"): 0.348,
    ("Production of electricity by petroleum and other oil derivatives", "PL"): 0.697,
    ("Production of electricity by petroleum and other oil derivatives", "PT"): 0.679,
    ("Production of electricity by petroleum and other oil derivatives", "RO"): 0.55,
    ("Production of electricity by petroleum and other oil derivatives", "RU"): 1.05,
    ("Production of electricity by petroleum and other oil derivatives", "SE"): 0.86,
    ("Production of electricity by petroleum and other oil derivatives", "SI"): 0.871,
    ("Production of electricity by petroleum and other oil derivatives", "SK"): 0,
    ("Production of electricity by petroleum and other oil derivatives", "TR"): 0.667,
    ("Production of electricity by petroleum and other oil derivatives", "US"): 0.891,
    ("Production of electricity by petroleum and other oil derivatives", "WA"): 0.66,
    ("Production of electricity by petroleum and other oil derivatives", "WE"): 0.761,
    ("Production of electricity by petroleum and other oil derivatives", "WF"): 0.77,
    ("Production of electricity by petroleum and other oil derivatives", "WL"): 0.81,
    ("Production of electricity by petroleum and other oil derivatives", "WM"): 0.977,
    ("Production of electricity by petroleum and other oil derivatives", "ZA"): 0.844,

    # Production of electricity by biomass and waste
    ("Production of electricity by biomass and waste", "AT"): 0.394,
    ("Production of electricity by biomass and waste", "AU"): 0.447,
    ("Production of electricity by biomass and waste", "BE"): 0.0407,
    ("Production of electricity by biomass and waste", "BG"): 0.307,
    ("Production of electricity by biomass and waste", "BR"): 0.415,
    ("Production of electricity by biomass and waste", "CA"): 0.0589,
    ("Production of electricity by biomass and waste", "CH"): 0.338,
    ("Production of electricity by biomass and waste", "CN"): 0.0275,
    ("Production of electricity by biomass and waste", "CZ"): 0.528,
    ("Production of electricity by biomass and waste", "DE"): 0.362,
    ("Production of electricity by biomass and waste", "DK"): 1.08,
    ("Production of electricity by biomass and waste", "EE"): 0.00885,
    ("Production of electricity by biomass and waste", "ES"): 0.121,
    ("Production of electricity by biomass and waste", "FI"): 0.108,
    ("Production of electricity by biomass and waste", "FR"): 0.52,
    ("Production of electricity by biomass and waste", "GB"): 0.189,
    ("Production of electricity by biomass and waste", "GR"): 0.579,
    ("Production of electricity by biomass and waste", "HR"): 0.562,
    ("Production of electricity by biomass and waste", "HU"): 0.0777,
    ("Production of electricity by biomass and waste", "ID"): 0.0125,
    ("Production of electricity by biomass and waste", "IE"): 0.182,
    ("Production of electricity by biomass and waste", "IN"): 0.072,
    ("Production of electricity by biomass and waste", "IT"): 0.205,
    ("Production of electricity by biomass and waste", "JP"): 0.0225,
    ("Production of electricity by biomass and waste", "KR"): 0.0594,
    ("Production of electricity by biomass and waste", "MX"): 0.153,
    ("Production of electricity by biomass and waste", "NL"): 0.0437,
    ("Production of electricity by biomass and waste", "NO"): 0.312,
    ("Production of electricity by biomass and waste", "PL"): 0.0176,
    ("Production of electricity by biomass and waste", "PT"): 0.176,
    ("Production of electricity by biomass and waste", "RO"): 0.0989,
    ("Production of electricity by biomass and waste", "SI"): 1.01,
    ("Production of electricity by biomass and waste", "SK"): 0.52,
    ("Production of electricity by biomass and waste", "TR"): 0.516,
    ("Production of electricity by biomass and waste", "US"): 0.332,
    ("Production of electricity by biomass and waste", "WA"): 0.0727,
    ("Production of electricity by biomass and waste", "WE"): 0.331,
    ("Production of electricity by biomass and waste", "WF"): 0.0493,
    ("Production of electricity by biomass and waste", "WL"): 0.0505,
    ("Production of electricity by biomass and waste", "WM"): 0.473,
    ("Production of electricity by biomass and waste", "ZA"): 0.0273,
}

In [47]:
df["direct emission factor"] = df.apply(lambda x: direct_emission_dict.get((x["description"], x["region_code"]), 0), axis=1)

In [48]:
df["indirect emission factor"] = df["emission factor"] - df["direct emission factor"]

#sort
df = df[['description', 'region_code', 'flow_code', 'id','unit', 'emission factor', 'direct emission factor', 'indirect emission factor', ]]

In [49]:
df

,description,region_code,flow_code,id,unit,emission factor,direct emission factor,indirect emission factor
0,Production of electricity by coal,AT,A_POWC,32552,kg CO2/kWh,0.762699,0.757,0.005699
1,Production of electricity by coal,BE,A_POWC,32552,kg CO2/kWh,0.762699,0.928,-0.165301


In [70]:
#save to excel
df.to_csv("../data/bonsai_emission_factors.csv", sep = ";",  index=False)

### just to demonstrate how to retrieve a footprint when you had the footprint id.

In [46]:
#retrieve all ids of the electricity producing activities
id_list = []
for item in all_footprints:
    if item['flow_code'] in list_abbreviation:
        id_list.append(item['id'])

print(len(id_list))

338


In [ ]:
container = []

for id in id_list:
    url = f"http://lca.aau.dk/api/footprint/{id}/"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        container.append(data)
        
    else:
        print(f"Failed. Status code: {response.status_code}")
        print("Error message:", response.text)
        break